In [ ]:
# Clone github repository
!git clone --branch personal-contribution https://github.com/AlessandroMaini/federated-learning-project.git

In [ ]:
%cd federated-learning-project

In [ ]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import random
from data.cifar100_loader import get_unbalanced_cifar100_datasets
from eval import evaluate
from train import train, train_steps
from models.prepare_model import get_dino_vits16_model, freeze_backbone, unfreeze_backbone, freeze_head, unfreeze_head
from models.unbalance import compute_kl_discrepancy, compute_discrepancy_aware_weights_sigmoid
from models.model_editing import mask_calculator, freeze_and_clean_client_masks
from models.federated_averaging import train_on_client, average_metrics, average_models, get_trainable_keys
from tqdm import tqdm

In [ ]:
baseline = True

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
CHECKPOINT_DIR = './checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

In [ ]:
# Number of clients
K = 45
# Fraction of clients
C = 0.2
# Number of local steps
J = 64

In [ ]:
# Create dataloaders for the clients
train_datasets, test_loader, client_class_map, client_metadata = get_unbalanced_cifar100_datasets(K) # Default division by size and bias

criterion = nn.CrossEntropyLoss()

In [ ]:
# Compute KL divergence for each client
discrepancies = []
for client_dataset in train_datasets:
    kl_div = compute_kl_discrepancy(client_dataset)
    discrepancies.append(kl_div)

In [ ]:
client_sizes = [len(dataset) for dataset in train_datasets]
weights_disc = compute_discrepancy_aware_weights_sigmoid (client_sizes, discrepancies, tau=0.1)

In [ ]:
# Plot client sizes
plt.figure(figsize=(10, 5))
plt.bar(range(K), client_sizes, color='blue', alpha=0.7)
plt.xlabel('Client ID')
plt.ylabel('Number of samples')
plt.title('Number of samples per client')
plt.xticks(range(K), rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Plot client discrepancies
plt.figure(figsize=(10, 5))
plt.bar(range(K), discrepancies, color='orange', alpha=0.7)
plt.xlabel('Client ID')
plt.ylabel('Discrepancy')
plt.title('Discrepancy per client')
plt.xticks(range(K), rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Plot client weights
plt.figure(figsize=(10, 5))
plt.bar(range(K), weights_disc, color='green', alpha=0.7)
plt.xlabel('Client ID')
plt.ylabel('Client weight')
plt.title('Client weights')
plt.xticks(range(K), rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_client_scatter_weights_sizes_discrepancies(client_weights, discrepancies, client_sizes):
    """
    Scatter plot of client size vs discrepancy, with weight shown by color and point size.
    Y-axis is limited to [0, 1].
    """
    client_weights = np.array(client_weights)
    discrepancies = np.array(discrepancies)
    client_sizes = np.array(client_sizes)

    # Normalize weight for marker size
    point_sizes = 300 * (client_weights / client_weights.max())
    point_colors = client_weights

    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(client_sizes, discrepancies, c=point_colors, s=point_sizes,
                          cmap='viridis', edgecolors='k', alpha=0.8)

    plt.colorbar(scatter, label='Client Weight')
    plt.xlabel('Client Size')
    plt.ylabel('Discrepancy')
    plt.title('Client Size vs Discrepancy (Color & Size = Weight)')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

plot_client_scatter_weights_sizes_discrepancies(weights_disc,
                                                discrepancies,
                                                client_sizes)

In [ ]:
collaborative_model = get_dino_vits16_model(device)

In [ ]:
start_round = 0
num_rounds = 30
best_test_acc = 0.0

hist_train_loss = []
hist_train_acc = []
hist_test_loss = []
hist_test_acc = []

random.seed(42)

In [ ]:
# FedAvg loop
for round in range(start_round, start_round + num_rounds):
    print(f"\n--- Round {round + 1}/{start_round + num_rounds} ---")

    # Select clients
    selected_clients = random.sample(range(K), int(C * K))

    # Local training
    local_models, train_losses, train_accs = [], [], []
    for client_id in selected_clients:

        steps = max(1, int(J * (len(train_datasets[client_id]) / 1000)))

        model_state, loss, acc = train_on_client(
            client_id,
            collaborative_model,
            train_datasets[client_id],
            J,
            criterion,
            lr = 0.01,
            device = device,
        )
        local_models.append(model_state)
        train_losses.append(loss)
        train_accs.append(acc)

    # Weighting by dataset size for metrics
    client_sample_counts = [len(train_datasets[c]) for c in selected_clients]
    total_samples = sum(client_sample_counts)
    client_weights_metrics = [count / total_samples for count in client_sample_counts]

    if baseline:
        # Weighting uniformly
        client_weights = [1.0 / len(selected_clients)] * len(selected_clients)
    else:
        # Use computed client weights
        tot_weights_disc = sum(weights_disc[c] for c in selected_clients)
        client_weights = [weights_disc[c] / tot_weights_disc for c in selected_clients]

    # Federated averaging
    trainable_keys = get_trainable_keys(collaborative_model)
    averaged_state = average_models(local_models, client_weights, trainable_keys)
    new_state = collaborative_model.state_dict()
    for key in averaged_state:
        new_state[key] = averaged_state[key]
    collaborative_model.load_state_dict(new_state)

    # Log average training metrics
    avg_train_loss = average_metrics(train_losses, client_weights_metrics)
    avg_train_acc = average_metrics(train_accs, client_weights_metrics)
    print(f"Avg Train Loss: {avg_train_loss:.4f}, Avg Train Accuracy: {avg_train_acc:.4f}")
    hist_train_loss.append(avg_train_loss)
    hist_train_acc.append(avg_train_acc)

    if (round + 1) % 2 == 0:
        avg_test_loss, avg_test_acc = evaluate(collaborative_model, test_loader, criterion, device)

        print(f"Avg Test Loss: {avg_test_loss:.4f}, Avg Test Accuracy: {avg_test_acc:.4f}")
        hist_test_loss.append(avg_test_loss)
        hist_test_acc.append(avg_test_acc)

    # Save model each 20 rounds
    if (round + 1) % 10 == 0:
        checkpoint_path = os.path.join(CHECKPOINT_DIR, f"federated_unbalance_disco_round_{round + 1}.pth")
        torch.save(collaborative_model.state_dict(), checkpoint_path)

In [ ]:
# X-axis for train loss: computed every round
rounds_train = range(len(hist_train_loss))

# X-axis for test loss: computed every 5 rounds
rounds_test = range(1, (len(hist_test_loss) + 1) * 2, 2)

# Plotting
plt.plot(rounds_train, hist_train_loss, label='Train Loss')
plt.plot(rounds_test, hist_test_loss, label='Test Loss')
plt.xlabel('Rounds')
plt.ylabel('Loss')
plt.legend()

# Save image as vector format
plt.savefig(f"fed_unbalance_disco_loss.svg", format='svg', bbox_inches='tight')

plt.show()

In [ ]:
# X-axis for train loss: computed every round
rounds_train = range(len(hist_train_acc))

# X-axis for test loss: computed every 5 rounds
rounds_test = range(1, (len(hist_test_acc) + 2) * 2, 2)

# Plotting
plt.plot(rounds_train, hist_train_acc, label='Train Accuracy')
plt.plot(rounds_test, hist_test_acc, label='Test Accuracy')
plt.xlabel('Rounds')
plt.ylabel('Accuracy')
plt.legend()

# Save image as vector format
plt.savefig(f"fed_unbalance_disco_acc.svg", format='svg', bbox_inches='tight')

plt.show()